# Iso-Caller for scFASTseq data via Salmon-Alevin
#### This pipeline should be run AFTER the regular seeksoultools fast run!

In [ ]:
import os
import sys
import gzip
import shutil
import json
import subprocess
import re
from datetime import datetime

# Timestamp for logging
print(f"[{datetime.now().strftime('%H:%M:%S')}] Pipeline initialized.")
print(f"Python Version: {sys.version.split()[0]}")

## Ref links:
#### https://ftp.ensembl.org/pub/release-115/fasta/mus_musculus/cdna/ - cDNA Mus_musculus.GRCm39.cdna.all.fa.gz
#### https://app.flow.bio/data/657359233329314859/ - Mus_musculus.GRCm39.109.sorted.gtf

In [ ]:
# CONFIGURATION

# 1. Input Data (Raw FASTQ files)
FASTQ_R1 = 'path/to/FASTQ_R1.fastq.gz'
FASTQ_R2 = 'path/to/FASTQ_R2.fastq.gz'

matrix_type = 'filtered' # raw / filtered

# 2. Outpit directory
OUTPUT_ROOT = f"./Experiment_Isoform_{matrix_type}"

# 3. Reference Genome Assets
# cDNA FASTA is required for index building and tgMap generation
REF_FASTA = './iso_ref/Mus_musculus.GRCm39.cdna.all.fa.gz'
# GTF is required for final feature renaming
REF_GTF = './iso_ref/Mus_musculus.GRCm39.109.sorted.gtf'

# 4. Barcode Whitelist
# Path to the list of valid cellular barcodes (from a seeksoultools fast run or any other list)
# Note: Use the 'raw' list if you intend to use SoupX downstream.
BARCODE_WHITELIST_GZ = f'path/to/seeksoultools-fast-output/step3/{matrix_type}_feature_bc_matrix/barcodes.tsv.gz'

# 5. Library Geometry (SeekOne DD)
# Geometry: Barcode (17bp) + UMI (12bp) located at the 5' end of Read 1.
BARCODE_LEN = 17
UMI_LEN = 12
READ_ORIENTATION = "5" # Barcodes are at the beginning of the read
LIB_TYPE = "ISR"       # Inward Stranded Reverse (Read 2 matches transcript sense)

# 6. System Resources
CPU_THREADS = 8

# Directory Initialization
SALMON_DIR = os.path.join(OUTPUT_ROOT, "salmon")
FINAL_MATRIX_DIR = os.path.join(OUTPUT_ROOT, "raw_isoform_matrix")

os.makedirs(SALMON_DIR, exist_ok=True)
os.makedirs(FINAL_MATRIX_DIR, exist_ok=True)

print(f"[{datetime.now().strftime('%H:%M:%S')}] Configuration loaded.")
print(f"Output Directory: {os.path.abspath(OUTPUT_ROOT)}")

In [ ]:
def log(message):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {message}")

def ensure_salmon_installed():
    """
    Robustly checks for or installs Salmon. 
    Handles Linux servers and macOS nuances automatically.
    """
    # 1. Check existing installation
    salmon_exec = shutil.which("salmon")
    if salmon_exec:
        try:
            res = subprocess.run([salmon_exec, "--version"], capture_output=True, text=True)
            log(f"System Salmon detected: {res.stdout.strip()}")
            return salmon_exec
        except Exception:
            pass

    log("Salmon not found in PATH. Initiating installation routine...")
    system_os = platform.system()
    
    # STRATEGY A: CONDA (Cross-Platform)
    if shutil.which("conda"):
        log("Attempting installation via Conda (ensuring conda-forge for dependencies)...")
        try:
            # Explicitly adding conda-forge fixes libiconv issues on Mac M1/M2
            cmd = ["conda", "install", "-c", "conda-forge", "-c", "bioconda", "salmon>=1.10.0", "-y"]
            subprocess.run(cmd, check=True)
            
            salmon_exec = shutil.which("salmon")
            if salmon_exec:
                log(f"Salmon installed via Conda at: {salmon_exec}")
                return salmon_exec
        except subprocess.CalledProcessError as e:
            log(f"Conda installation failed: {e}")

    # STRATEGY B: LINUX BINARY DOWNLOAD (Linux Only)
    if system_os == "Linux":
        log("Downloading pre-compiled binary from GitHub...")
        url = "https://github.com/COMBINE-lab/salmon/releases/download/v1.10.0/salmon-1.10.0_linux_x86_64.tar.gz"
        filename = "salmon-1.10.0_linux_x86_64.tar.gz"
        folder = "salmon-1.10.0_linux_x86_64"
        
        # Use wget or curl
        if shutil.which("wget"):
            subprocess.run(["wget", "-q", "-O", filename, url], check=True)
        else:
            subprocess.run(["curl", "-L", "-o", filename, url], check=True)
            
        subprocess.run(["tar", "xzf", filename], check=True)
        local_bin = os.path.abspath(f"{folder}/bin/salmon")
        
        if os.path.exists(local_bin):
            log(f"Salmon installed locally at: {local_bin}")
            return local_bin

    # STRATEGY C: HOMEBREW (macOS Only)
    if system_os == "Darwin" and shutil.which("brew"):
        log("Attempting installation via Homebrew...")
        try:
            subprocess.run(["brew", "tap", "brewsci/bio"], check=True)
            subprocess.run(["brew", "install", "salmon"], check=True)
            salmon_exec = shutil.which("salmon")
            if salmon_exec:
                log("Salmon installed via Homebrew.")
                return salmon_exec
        except subprocess.CalledProcessError:
            pass

    raise FileNotFoundError("CRITICAL: Failed to install Salmon. Please install 'salmon' manually in your terminal.")

# Initialize
salmon_path = ensure_salmon_installed()

In [ ]:
INDEX_DIR = os.path.join(SALMON_DIR, "salmon_index")
TGMAP_FILE = os.path.join(SALMON_DIR, "isoform_tgmap.tsv")

# A. Build Index
if os.path.exists(os.path.join(INDEX_DIR, "pos.bin")):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Existing Salmon index found. Skipping build.")
else:
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Building Salmon index (this may take time)...")
    cmd = [
        salmon_path, "index",
        "-t", REF_FASTA,
        "-i", INDEX_DIR,
        "-p", str(CPU_THREADS)
    ]
    subprocess.run(cmd, check=True)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Indexing complete.")

# B. Generate tgMap (Identity Mapping)
print(f"[{datetime.now().strftime('%H:%M:%S')}] Generating transcript-to-gene map from FASTA headers...")

tx_count = 0
open_func = gzip.open if REF_FASTA.endswith('.gz') else open
mode = 'rt' if REF_FASTA.endswith('.gz') else 'r'

with open_func(REF_FASTA, mode) as f_in, open(TGMAP_FILE, 'w') as f_out:
    for line in f_in:
        if line.startswith(">"):
            # Extract ID (e.g., >ENSMUST000...2 cdna...) -> ENSMUST000...2
            # We map ID -> ID to force Alevin to quantify isoforms individually.
            tx_id = line.split()[0].replace(">", "")
            f_out.write(f"{tx_id}\t{tx_id}\n")
            tx_count += 1

print(f"[{datetime.now().strftime('%H:%M:%S')}] tgMap generated containing {tx_count} entries.")

In [ ]:
WHITELIST_TSV = os.path.join(SALMON_DIR, "whitelist.tsv")

if os.path.exists(BARCODE_WHITELIST_GZ):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Decompressing whitelist: {os.path.basename(BARCODE_WHITELIST_GZ)}")
    with gzip.open(BARCODE_WHITELIST_GZ, 'rb') as f_in, open(WHITELIST_TSV, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
else:
    raise FileNotFoundError(f"Whitelist not found at: {BARCODE_WHITELIST_GZ}")

In [ ]:
ALEVIN_OUT_DIR = os.path.join(SALMON_DIR, "alevin_output")

# Cleanup previous runs to ensure data integrity
if os.path.exists(ALEVIN_OUT_DIR):
    shutil.rmtree(ALEVIN_OUT_DIR)

cmd = [
    salmon_path, "alevin",
    "-l", LIB_TYPE,             # Library type (ISR)
    "-1", FASTQ_R1,             # Read 1 (Barcodes)
    "-2", FASTQ_R2,             # Read 2 (Biological)
    "-i", INDEX_DIR,            # Index
    "-p", str(CPU_THREADS),
    "-o", ALEVIN_OUT_DIR,
    "--tgMap", TGMAP_FILE,
    "--whitelist", WHITELIST_TSV,
    "--dumpMtx",                # Export sparse matrix
    # Explicit Geometry Definition
    "--barcodeLength", str(BARCODE_LEN),
    "--umiLength", str(UMI_LEN),
    "--end", READ_ORIENTATION
]

print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting Salmon Alevin Quantification...")
print(f"Command execution: {' '.join(cmd)}\n")

# Execute with real-time log streaming
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Stream logs
for line in process.stdout:
    print(line, end="")

process.wait()

if process.returncode == 0:
    print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Quantification finished successfully.")
else:
    raise RuntimeError(f"Salmon Alevin terminated with error code {process.returncode}.")

In [ ]:
print(f"[{datetime.now().strftime('%H:%M:%S')}] Processing output matrix and annotations...")

# 1. Load Annotations (GTF)
id_to_readable = {}
open_func = gzip.open if REF_GTF.endswith('.gz') else open
mode = 'rt' if REF_GTF.endswith('.gz') else 'r'

with open_func(REF_GTF, mode) as f:
    for line in f:
        if line.startswith("#") or "\ttranscript\t" not in line: 
            continue
        attrs = {}
        for item in line.strip().split("\t")[8].split(";"):
            if not item.strip(): continue
            parts = item.strip().split(" ")
            if len(parts) >= 2:
                key = parts[0].strip()
                val = parts[1].replace('"', '').replace(';', '').strip()
                attrs[key] = val
        tx_id = attrs.get("transcript_id")
        tx_name = attrs.get("transcript_name", tx_id)
        if tx_id:
            id_to_readable[tx_id.split('.')[0]] = tx_name

print(f"[{datetime.now().strftime('%H:%M:%S')}] Loaded annotations for {len(id_to_readable)} transcripts.")

# 2. Locate Files
found_matrix_path = None
real_source_dir = None
is_matrix_gzipped = False

for root, dirs, files in os.walk(SALMON_DIR):
    if "quants_mat.mtx.gz" in files:
        found_matrix_path = os.path.join(root, "quants_mat.mtx.gz")
        real_source_dir = root
        is_matrix_gzipped = True
        break
    elif "quants_mat.mtx" in files:
        found_matrix_path = os.path.join(root, "quants_mat.mtx")
        real_source_dir = root
        is_matrix_gzipped = False
        break

if not found_matrix_path:
    print(f"Could not find matrix file in: {SALMON_DIR}")
    raise FileNotFoundError("Salmon output matrix not found.")

print(f"[{datetime.now().strftime('%H:%M:%S')}] Found matrix: {found_matrix_path}")

# 3. Process & Transpose

# A. MATRIX (Needs Transpose: Cells x Genes -> Genes x Cells)
target_mtx = os.path.join(FINAL_MATRIX_DIR, "matrix.mtx.gz")
print("   -> Transposing and compressing matrix (Cells x Genes => Genes x Cells)...")

read_mode = 'rt' if is_matrix_gzipped else 'r'
open_in = gzip.open if is_matrix_gzipped else open

with open_in(found_matrix_path, read_mode) as f_in, gzip.open(target_mtx, 'wt') as f_out:
    header_processed = False
    for line in f_in:
        # Skip comments
        if line.startswith('%'):
            f_out.write(line)
            continue
        
        parts = line.split()
        if not header_processed:
            # Dimensions line: Rows Cols NonZeros
            # Alevin: Rows=Cells, Cols=Genes
            # Seurat wants: Rows=Genes, Cols=Cells
            # ACTION: Swap dimensions
            if len(parts) >= 2:
                rows, cols = parts[0], parts[1]
                rest = parts[2:]
                f_out.write(f"{cols} {rows} {' '.join(rest)}\n")
                header_processed = True
        else:
            # Data line: Row Col Val
            # ACTION: Swap Row and Col indices
            if len(parts) >= 3:
                r, c = parts[0], parts[1]
                val = parts[2]
                f_out.write(f"{c} {r} {val}\n")

# B. BARCODES (From ROWS.txt)
# Alevin Rows = Cells -> Barcodes.tsv
rows_src = os.path.join(real_source_dir, "quants_mat_rows.txt")
if not os.path.exists(rows_src): rows_src += ".gz"

target_barcodes = os.path.join(FINAL_MATRIX_DIR, "barcodes.tsv.gz")
print("   -> Processing barcodes (from rows)...")

if os.path.exists(rows_src):
    # Just copy/compress, raw barcodes are usually fine
    is_gz = rows_src.endswith('.gz')
    with (gzip.open if is_gz else open)(rows_src, 'rb') as f_in, gzip.open(target_barcodes, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
else:
    print("Warning: Rows file (Barcodes) not found!")

# C. FEATURES (From COLS.txt)
# Alevin Cols = Genes -> Features.tsv
cols_src = os.path.join(real_source_dir, "quants_mat_cols.txt")
if not os.path.exists(cols_src): cols_src += ".gz"

target_features = os.path.join(FINAL_MATRIX_DIR, "features.tsv.gz")
print("   -> Processing features (from cols)...")

if os.path.exists(cols_src):
    is_gz = cols_src.endswith('.gz')
    read_mode = 'rt' if is_gz else 'r'
    open_mode = gzip.open if is_gz else open
    
    with open_mode(cols_src, read_mode) as f_in, gzip.open(target_features, 'wt') as f_out:
        for line in f_in:
            tx_id_full = line.strip() 
            tx_id_base = tx_id_full.split('.')[0]
            display_name = id_to_readable.get(tx_id_base, tx_id_full)
            # Format: ID \t Name \t Type
            f_out.write(f"{tx_id_full}\t{display_name}\tTranscript Expression\n")
else:
    print("Warning: Cols file (Features) not found!")

print(f"[{datetime.now().strftime('%H:%M:%S')}] Pipeline finished! Output: {os.path.abspath(FINAL_MATRIX_DIR)}")